# AnimateDiff-V2V-GUI

I have forked AnimateDiff prompt travel to add front GUI
to do that, actually I have updated all those folder names etc

What you need is slitely different.
I'm keep syncing the repo so far and hopefully those basic function can also be available.
(But result folder structure is a bit different)


### How is it?

[AnimateDiff with prompt travel](https://github.com/s9roll7/animatediff-cli-prompt-travel) + [ControlNet](https://github.com/lllyasviel/ControlNet) + [IP-Adapter](https://github.com/tencent-ailab/IP-Adapter)
And now you have GUI

### How to install

Python 3.10 and git client must be installed.

git clone the repo.

```shell

git clone https://github.com/JojoYay/animatediff-cli-prompt-travel
cd animatediff-cli-prompt-travel
py -3.10 -m venv venv
venv\Scripts\activate.bat
set PYTHONUTF8=1
python -m pip install --upgrade pip
# Torch installation must be modified to suit the environment. (https://pytorch.org/get-started/locally/)
python -m pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
python -m pip install -e .

```
### Model, Motion Module, Lora prep
You need to prepare Model, Motion Module, Lora and place them below

model : /animatediff-cli-prompt-travel/data/sd_models
LoRA : /animatediff-cli-prompt-travel/data/lora
model : /animatediff-cli-prompt-travel/data/motion_modules

when you start, you can see the URL in console.
Click and enjoy making video!

In [1]:
#!rm -r /notebooks/storage/aj/animatediff-cli-prompt-travel/stylize/real_base2-dance00004



#import onnxruntime as ort
#ort.get_device()
#ort.get_available_providers()
#!pip uninstall onnxruntime onnxruntime-gpu onnxruntime_gpu -y
#!pip install onnxruntime_gpu

import sys
sys.path.append('/storage/aj/animatediff-cli-prompt-travel/src')
print(sys.path)

#!cp /notebooks/storage/aj/animatediff-cli-prompt-travel/config/test/* /notebooks/storage/aj/animatediff-cli-prompt-travel/config/test2


['/storage/aj/animatediff-cli-prompt-travel', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/storage/aj/animatediff-cli-prompt-travel/src']


In [ ]:
import gradio as gr
from animatediff.execute import execute
from animatediff.front_utils import (get_schedulers, validate_inputs, getNow, download_video, create_file_list,
                                    find_safetensor_files, find_last_folder_and_mp4_file, find_next_available_number,
                                    find_and_get_composite_video, load_video_name, get_last_sorted_subfolder,
                                    create_config_by_gui, get_config_path, update_config, change_ip, change_ad, change_op,
                                    change_dp, change_la)
from animatediff.settings import ModelConfig, get_model_config
from animatediff.video_utils import create_video
import io
import os
import time
from pathlib import Path
import shutil


# Define the function signature
def execute_wrapper(
      url: str, 
      inp_model: str, inp_mm: str,
      inp_sche: str, inp_step: int, inp_cfg: float, 
      inp_posi: str, inp_neg: str, 
      inp_lora1: str, inp_lora1_step: float,
      inp_lora2: str, inp_lora2_step: float,
      inp_lora3: str, inp_lora3_step: float,
      inp_lora4: str, inp_lora4_step: float,
      ip_ch: bool, ip_image: str, ip_scale: float, ip_type: str,
      ad_ch: bool, ad_scale: float, op_ch: bool, op_scale: float,
      dp_ch: bool, dp_scale:float, la_ch: bool, la_scale: float,
      delete_if_exists: bool, is_test: bool, is_refine: bool,
      progress=gr.Progress(track_tqdm=True)):
    
    yield 'generation Initiated...', None, None, None, None, gr.Button("Generating...", scale=1, interactive=False)
    
    start_time = time.time()

    time_str = getNow()
    validate_inputs(url)

    bg_config = None
    save_folder = 'data/video'
    saved_file = download_video(url, save_folder)
    video_name=saved_file.rsplit('.', 1)[0].rsplit('/notebooks', 1)[-1].rsplit('/', 1)[-1]
    stylize_dir= Path('/storage/aj/animatediff-cli-prompt-travel/stylize/' + video_name)
    create_config_by_gui(
        now_str=time_str,
        video = saved_file,
        stylize_dir = stylize_dir, 
        model=inp_model, motion_module=inp_mm, 
        scheduler=inp_sche, step=inp_step, cfg=inp_cfg, 
        head_prompt=inp_posi, neg_prompt=inp_neg,
        inp_lora1=inp_lora1, inp_lora1_step=inp_lora1_step,
        inp_lora2=inp_lora2, inp_lora2_step=inp_lora2_step,
        inp_lora3=inp_lora3, inp_lora3_step=inp_lora3_step,
        inp_lora4=inp_lora4, inp_lora4_step=inp_lora4_step,
        ip_ch=ip_ch, ip_image=ip_image, ip_scale=ip_scale, ip_type=ip_type,
        ad_ch=ad_ch, ad_scale=ad_scale, op_ch=op_ch, op_scale=op_scale,
        dp_ch=dp_ch, dp_scale=dp_scale, la_ch=la_ch, la_scale=la_scale,
    )

    yield from execute_impl(now_str=time_str,video=saved_file, delete_if_exists=delete_if_exists, is_test=is_test, is_refine=is_refine, bg_config=bg_config)

    end_time = time.time()
    execution_time = end_time - start_time
    print(f"実行時間: {execution_time}秒")

def execute_impl(now_str:str,
                 video: str, 
                 # config: Path, 
                 delete_if_exists: bool, 
                 is_test: bool,
                 is_refine: bool, 
                 bg_config: str):
    if video.startswith("/notebooks"):
        video = video[len("/notebooks"):]
    if bg_config is not None:
        if bg_config.startswith("/notebooks"):
            bg_config = bg_config[len("/notebooks"):]
    print(f"video1: {video}")
    yield 'generating config...', video, None, None, None, gr.Button("Generating...", scale=1, interactive=False)
        
    video_name=video.rsplit('.', 1)[0].rsplit('/notebooks', 1)[-1].rsplit('/', 1)[-1]

    stylize_dir='stylize/' + video_name
    stylize_fg_dir = stylize_dir + '/fg_00_'+video_name
    stylize_fg_dir = Path(stylize_fg_dir)
    stylize_bg_dir = stylize_dir + '/bg_'+video_name
    stylize_bg_dir = Path(stylize_bg_dir)
    stylize_dir = Path(stylize_dir)
    print(f"stylize_dir:{stylize_dir}")
    print(f"stylize_fg_dir:{stylize_fg_dir}")

    if bg_config is not None:
        bg_config = Path(bg_config)
        bg_model_config: ModelConfig = get_model_config(bg_config)

    if stylize_dir.exists() and not delete_if_exists:
        print(f"config already exists. skip create-config")
    else:
        if stylize_dir.exists():
            print(f"Delete folder and create again")
            shutil.rmtree(stylize_dir)
        # create_config(org_movie=video,config_org=config,fps=15)
        !animatediff stylize create-config {video}
        # create_mask(stylize_dir=stylize_dir, bg_config=bg_config, no_crop=True)
        !animatediff stylize create-mask {stylize_dir} -nc

    update_config(now_str, stylize_dir, stylize_fg_dir)
    config = get_config_path(now_str)
    model_config: ModelConfig = get_model_config(config)       
        
    yield 'generating fg bg video...', video, None, None, None, gr.Button("Generating...", scale=1, interactive=False)

    if is_test:
  #      generate(stylize_dir=stylize_fg_dir, length=16)
        !animatediff stylize generate {stylize_fg_dir} -L 16
        if bg_config is not None:
            # generate(stylize_dir=stylize_bg_dir, length=16)
            !animatediff stylize generate {stylize_bg_dir} -L 16

    else:
        # generate(stylize_dir=stylize_fg_dir)
        !animatediff stylize generate {stylize_fg_dir}
        if bg_config is not None:
            # generate(stylize_dir=stylize_bg_dir)
            !animatediff stylize generate {stylize_bg_dir}
            
    video2 = find_last_folder_and_mp4_file(stylize_fg_dir)
    print(f"video2: {video2}")

    if is_refine:
        yield 'refining fg video', video, video2, None, None, gr.Button("Generating...", scale=1, interactive=False)

        result_dir = get_last_sorted_subfolder(get_last_sorted_subfolder(stylize_fg_dir))
#        refine(frames_dir=result_dir, out_dir=stylize_fg_dir, config_path=config, width=768)
        !animatediff refine {result_dir} -o {stylize_fg_dir} -c {config} -W 768
        video3 = find_last_folder_and_mp4_file(get_last_sorted_subfolder(stylize_fg_dir))
        print(f"video3: {video3}")
        yield 'compositing video', video, video2, video3, None, gr.Button("Generate Video", scale=1, interactive=False)
        fg_result = get_last_sorted_subfolder(get_last_sorted_subfolder(get_last_sorted_subfolder(stylize_fg_dir)))

    else:
        yield 'composite video', video, video2, None, None, gr.Button("Generating...", scale=1, interactive=False)
        fg_result = get_last_sorted_subfolder(get_last_sorted_subfolder(stylize_fg_dir))
        video3 = None

    bg_result = get_last_sorted_subfolder(stylize_bg_dir)

    print(f"fg_dir:{fg_result}")
    if bg_config is not None:
        print(f"bg_dir: {bg_result}")
    else:
        print(f"bg_dir: {stylize_bg_dir/'00_img2img'}")

    if bg_config is not None:
        # final_video_dir = composite(stylize_dir=stylize_dir, bg_dir=bg_result, fg_dir=fg_result)
        !animatediff stylize composite {stylize_dir} -bg {bg_result} -fg {fg_result}  
    else:
        bg_result = stylize_bg_dir/'00_img2img'
        # final_video_dir = composite(stylize_dir=stylize_dir, bg_dir=stylize_bg_dir/'00_img2img', fg_dir=fg_result)
        !animatediff stylize composite {stylize_dir} -bg {bg_result} -fg {fg_result}

    fg_result = find_and_get_composite_video(stylize_dir)
    print(f"final_video_dir: {fg_result}")

    final_dir = os.path.dirname(fg_result)
    new_file_path = os.path.join(final_dir,  video_name + ".mp4")
    
#    final_video_dir: stylize/dance00023/cp_2023-12-18_08-09/composite2023-12-18_08-09-41

    try:
        create_video(video, fg_result, new_file_path)
        print(f"new_file_path:{new_file_path}")
        yield 'video is ready!', video, video2, video3, new_file_path, gr.Button("Generate Video", scale=1, interactive=True)
    except Exception as e:
        print(f"error:{e}")
        yield 'video is ready!(no music added)', video, video2, video3, fg_result, gr.Button("Generate Video", scale=1, interactive=True)
        
def launch():
    result_list = create_file_list("config/fix")
    safetensor_files = find_safetensor_files("data/sd_models")
    lora_files = find_safetensor_files("data/lora")
    mm_files = find_safetensor_files("data/motion_modules")
    schedulers = get_schedulers()
    ip_choice = ["full_face", "plus_face", "plus", "light"]

    with gr.Blocks() as iface:
        with gr.Row():
            gr.Markdown(
                """
                # AnimateDiff-V2V-GUI
                """, scale=8)
            btn = gr.Button("Generate Video", scale=1)
        with gr.Row():
            with gr.Column():
                with gr.Group():
                    url = gr.Textbox(lines=1, value="https://www.tiktok.com/@ai_hinahina/video/7313863412541361426", placeholder="https://www.tiktok.com/@ai_hinahina/video/7313863412541361426", label="URL")
                    with gr.Group():
                        with gr.Row():
                            inp_model = gr.Dropdown(choices=safetensor_files, label="Model")
                            inp_mm = gr.Dropdown(choices=mm_files, label="Motion Module")
                    with gr.Group():
                        with gr.Row():
                            inp_sche = gr.Dropdown(choices=schedulers, label="Sampling Method")
                            inp_step = gr.Slider(minimum=1, maximum=20, step=1, value=9, label="Sampling Steps")
                            inp_cfg = gr.Slider(minimum=0.1, maximum=10, step=0.1,  value=2.3, label="CFG Scale")
                    inp_posi = gr.Textbox(lines=2, value="1girl, beautiful", placeholder="1girl, beautiful", label="Positive Prompt")
                    inp_neg = gr.Textbox(lines=2, value="low quality, low res,", placeholder="low quality, low res,", label="Negative Prompt")
                    with gr.Accordion("LoRAs", open=False):
                        with gr.Group():
                            with gr.Row():
                                inp_lora1 = gr.Dropdown(choices=lora_files, label="Lora1", scale=3)
                                inp_lora1_step = gr.Slider(minimum=0.1, maximum=3, step=0.1, value=1.0, label="LoRA1 Scale", scale=1)
                        with gr.Group():
                            with gr.Row():
                                inp_lora2 = gr.Dropdown(choices=lora_files, label="Lora2", scale=3)
                                inp_lora2_step = gr.Slider(minimum=0.1, maximum=3, step=0.1, value=1.0, label="LoRA2 Scale", scale=1)
                        with gr.Group():
                            with gr.Row():
                                inp_lora3 = gr.Dropdown(choices=lora_files, label="Lora3", scale=3)
                                inp_lora3_step = gr.Slider(minimum=0.1, maximum=3, step=0.1, value=1.0, label="LoRA3 Scale", scale=1)
                        with gr.Group():
                            with gr.Row():
                                inp_lora4 = gr.Dropdown(choices=lora_files, label="Lora4", scale=3)
                                inp_lora4_step = gr.Slider(minimum=0.1, maximum=3, step=0.1, value=1.0, label="LoRA4 Scale", scale=1)

                    with gr.Accordion("ControlNet", open=False):
                    # with gr.Group():
                        ip_ch = gr.Checkbox(label="IPAdapter", value=False)
                        ip_image = gr.Image(height=256, type="pil", interactive=False)
                        # ip_upload = gr.UploadButton(label='Click to uplaod Image', file_types=["image"], file_count="single")
                        with gr.Row():
                            ip_scale = gr.Slider(minimum=0, maximum=3, step=0.1, value=0.5, label="IPAdapter scale", interactive=False)
                            ip_type = gr.Radio(choices=ip_choice, label="IPAdapter Type", value="plus_face", interactive=False)

                    # with gr.Group():
                        with gr.Row():
                            ad_ch = gr.Checkbox(label="AimateDiff Controlnet", value=True)
                            ad_scale = gr.Slider(minimum=0, maximum=3,  step=0.1, value=0.8, label="AnimateDiff Controlnet scale")
                    # with gr.Group():
                        with gr.Row():
                            op_ch = gr.Checkbox(label="Open Pose", value=True)
                            op_scale = gr.Slider(minimum=0, maximum=3,  step=0.1, value=1.0, label="Open Pose scale")
                    # with gr.Group():
                        with gr.Row():
                            dp_ch = gr.Checkbox(label="Depth", value=False)
                            dp_scale = gr.Slider(minimum=0, maximum=3,  step=0.1, value=1.0, label="Depth scale", interactive=False)
                    # with gr.Group():
                        with gr.Row():
                            la_ch = gr.Checkbox(label="Lineart", value=False)
                            la_scale = gr.Slider(minimum=0, maximum=3,  step=0.1, value=1.0, label="Lineart scale", interactive=False)
                                
                 #   inp2 = gr.Dropdown(choices=result_list, info="please select", label="Config")
                    with gr.Row():
                        delete_if_exists = gr.Checkbox(label="Delete cache")
                        test_run = gr.Checkbox(label="Test Run", value=True)
                        refine = gr.Checkbox(label="Refine")
                    

            with gr.Column():
                with gr.Group():
                    o_status = gr.Label(value="Not Started Yet", label="Status")
                    with gr.Row():
                        o_video1 = gr.Video(width=256, label="Original Video", show_share_button=True)
                        o_video2 = gr.Video(width=256, label="Front Video", show_share_button=True)
                    with gr.Row():
                        o_video3 = gr.Video(width=256, label="Refined Front Video", show_share_button=True)
                        o_video4 = gr.Video(width=256, label="Generated Video", show_share_button=True)
        
        btn.click(fn=execute_wrapper,
                  inputs=[url, 
                          inp_model, inp_mm,
                          inp_sche, inp_step, inp_cfg, 
                          inp_posi, inp_neg, 
                          inp_lora1, inp_lora1_step,
                          inp_lora2, inp_lora2_step,
                          inp_lora3, inp_lora3_step,
                          inp_lora4, inp_lora4_step,
                          ip_ch, ip_image, ip_scale, ip_type,
                          ad_ch, ad_scale, op_ch, op_scale,
                          dp_ch, dp_scale, la_ch, la_scale,
                          delete_if_exists, test_run, refine],
                  outputs=[o_status, o_video1, o_video2, o_video3, o_video4, btn])

        ip_ch.change(fn=change_ip, inputs=[ip_ch], outputs=[ip_ch, ip_image, ip_scale, ip_type])        
        ad_ch.change(fn=change_ad, inputs=[ad_ch], outputs=[ad_ch, ad_scale])
        op_ch.change(fn=change_op, inputs=[op_ch], outputs=[op_ch, op_scale])
        dp_ch.change(fn=change_dp, inputs=[dp_ch], outputs=[dp_ch, dp_scale])
        la_ch.change(fn=change_la, inputs=[la_ch], outputs=[la_ch, la_scale])

        
    iface.queue()
    iface.launch(share=True)

    while True:
        pass

launch()


/usr/local/lib/python3.10/dist-packages/controlnet_aux/segment_anything/modeling/tiny_vit_sam.py:654: UserWarning: Overwriting tiny_vit_5m_224 in registry with controlnet_aux.segment_anything.modeling.tiny_vit_sam.tiny_vit_5m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  return register_model(fn_wrapper)
/usr/local/lib/python3.10/dist-packages/controlnet_aux/segment_anything/modeling/tiny_vit_sam.py:654: UserWarning: Overwriting tiny_vit_11m_224 in registry with controlnet_aux.segment_anything.modeling.tiny_vit_sam.tiny_vit_11m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  return register_model(fn_wrapper)
/usr/local/lib/python3.10/dist-packages/controlnet_aux/segment_anything/modeling/tiny_vit_sam.py:654: UserWarning: Overwriting tiny_vit_21m_224 in registry with controlnet_aux.segment_anything.modeling.tiny_vit_sam.tiny_vit_21m_224. This 

diffuser_ver='0.23.0'


/usr/local/lib/python3.10/dist-packages/flax/struct.py:136: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)
/usr/local/lib/python3.10/dist-packages/flax/struct.py:136: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)


Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://3de434e035bcce1a0c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


[TikTok] Extracting URL: https://www.tiktok.com/@namdareun_/video/7309769737024605447
[TikTok] 7309769737024605447: Downloading video feed
[info] 7309769737024605447: Downloading 1 format(s): bytevc1_1080p_1730203-2
[download] data/video/dance00032.mp4 has already been downloaded
[download] 100% of    7.72MiB
inp_posi(masterpiece, best quality, detailed), 1girl, solo, indoors, changing room, arms behind head, cowboy shot, souryuu asuka langley, interface headset, red bodysuit
inp_lora1lora/sd/Asuka.safetensors
inp_lora1_step1
neg_prompt(low quality, worst quality:1.4), signature, bad-hands-5, EasyNegativeV2, multicolored bodysuit
<PIL.Image.Image image mode=RGB size=1024x1536 at 0x7F95A7739FC0>
lora/sd/Asuka.safetensors
########################################################
Image saved successfully to /storage/aj/animatediff-cli-prompt-travel/stylize/dance00032/00_ipadapter/0.png
########################################################
video1: data/video/dance00032.mp4
stylize_dir:st

In [2]:
# Optional path settings
repo_storage_dir_aj = '/storage/aj'         # Where to store your animatediff-cli-prompt-travel-related files.

repo_dir = '/notebooks'    
!ln -s /storage/ /notebooks/
!ln -s /tmp/ /notebooks

%store repo_dir 
# ===================================================================================================
# Save variables to Jupiter's temp storage so we can access it even if the kernel restarts.
%store repo_storage_dir_aj

#+++GIT更新++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
import os
from pathlib import Path

repo_storage_dir_aj = Path(repo_storage_dir_aj)
anime_diff_path = repo_storage_dir_aj / 'animatediff-cli-prompt-travel'

%cd "{Path(repo_storage_dir_aj, 'animatediff-cli-prompt-travel')}"
#!sudo apt-get update
#!sudo apt-get install build-essential git yasm cmake libtool libx265-dev -y

!pip install --upgrade pip
!pip uninstall onnxruntime onnxruntime-gpu onnxruntime_gpu -y
!pip install onnxruntime_gpu
#!source venv/bin/activate
!pip install -e .
!pip install -e .[all]

import sys
#sys.path.append('/storage/aj/animatediff-cli-prompt-travel/src')
print(sys.path)

# from animatediff.front import launch
# launch()



ln: failed to create symbolic link '/notebooks/storage': File exists
ln: failed to create symbolic link '/notebooks/tmp': File exists
Stored 'repo_dir' (str)
Stored 'repo_storage_dir_aj' (str)
/storage/aj/animatediff-cli-prompt-travel
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 77.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.0.1
    Uninstalling pip-23.0.1:
      Successfully uninstalled pip-23.0.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 94.0 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 74.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.1/157.1 MB 24.0 MB/s eta 0:00:0000:010:04m
Obtaining file:///storage/aj/animatediff-cli-prompt-travel
  Installing build dependencies ... done
  Checking if build b

In [ ]:
from animatediff.front import launch
launch()


compositing  89% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━ 16/18  [ 0:00:06 < 0:00:01 , 2 it/s ]

composite fg_array is None -> skip


Creating ffmpeg encoder...
Encoding interpolated frames with ffmpeg...


ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

output to stylize/dance00027/cp_2023-12-17_03-03/composite2023-12-17_03-14-09


frame=   18 fps=0.0 q=-1.0 Lsize=     422kB time=00:00:01.00 bitrate=3459.6kbits/s speed=3.89x    
video:421kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.233855%
[libx264 @ 0x56487e6be100] frame I:2     Avg QP: 6.32  size: 28988
[libx264 @ 0x56487e6be100] frame P:5     Avg QP:10.35  size: 30319
[libx264 @ 0x56487e6be100] frame B:11    Avg QP:12.16  size: 20109
[libx264 @ 0x56487e6be100] consecutive B-frames: 22.2%  0.0% 16.7%  0.0% 27.8% 33.3%
[libx264 @ 0x56487e6be100] mb I  I16..4: 45.0% 40.2% 14.9%
[libx264 @ 0x56487e6be100] mb P  I16..4:  4.7% 26.1%  8.9%  P16..4: 10.6% 14.2%  8.1%  0.0%  0.0%    skip:27.2%
[libx264 @ 0x56487e6be100] mb B  I16..4:  2.5%  5.9%  2.5%  B16..8: 27.7% 23.9%  6.6%  direct: 9.6%  skip:21.3%  L0:49.3% L1:26.9% BI:23.8%
[libx264 @ 0x56487e6be100] 8x8 transform intra:53.2% inter:61.5%
[libx264 @ 0x56487e6be100] coded y,uvDC,uvAC intra: 57.6% 73.0% 71.5% inter: 42.1% 60.3% 35.1%
[libx264 @ 0x56487e6be100] i16 v,h,dc,p: 72% 

fg_フォルダ: stylize/dance00027/fg_00_dance00027/2023-12-17_02-22_00/00-2023-12-17_02-26
bg_フォルダ: stylize/dance00027/bg_dance00027/00_img2img
final_video_dir: stylize/dance00027/cp_2023-12-17_03-03/composite2023-12-17_03-14-09


In [5]:
import time
start_time = time.time()

############################################################
#ここにビデオのURLを入力#######################################
urls = [
#    "https://www.tiktok.com/@assen047/video/7273517043876433158",   #dance00010
#    "https://www.tiktok.com/@xiavigor__/video/7292363704266427653", #dance00011
#    "https://www.tiktok.com/@admin_break/video/7306911339673324806", #dance00012
#    "https://www.tiktok.com/@admin_break/video/7309132061330181381",#dance00013
#    "https://www.tiktok.com/@xntonio9/video/7266524869783784737",   #dance00014 soccer
#    "https://www.tiktok.com/@chiranjit_babu/video/7309784153761795329", #dance00015 indo dance ３人でるから没
 #   "https://twitter.com/TaichiZhe/status/1732774857807646772", #dance00016 kick
 #   "https://twitter.com/tongbeijapan/status/1731906333081841755",#dance00017 punch2
 #   "https://www.tiktok.com/@chiranjit_babu/video/7287774575062945042",#dance00018 indo dance2
 #   "https://www.tiktok.com/@chiranjit_babu/video/7210939347212700936", #dance00019 indo dance3    
#    "https://www.tiktok.com/@fantasista283/video/7309087809829162247", #dance00020 soccer    
#    "https://www.tiktok.com/@lia.lewis/video/7297191511010118945", #dance00021 soccer girl
#    "https://www.tiktok.com/@raynavallandingham/video/7309946333593439530", #dance00022 girl kunfoo
    "https://www.tiktok.com/@fukada0318/video/7290017445211819266", #dance00023 eimi fukada
#    "https://www.tiktok.com/@ai_hinahina/video/7312648648553204999", #dance00027
]
videos = [
#    '/notebooks/storage/aj/animatediff-cli-prompt-travel/stylize/Santa-dance00004',
#    '/notebooks/storage/animatediff/animatediff-cli-prompt-travel/data/video/dance31.mp4'
#    '/notebooks/storage/animatediff/animatediff-cli-prompt-travel/data/video/dance32.mp4',
#    '/notebooks/storage/animatediff/animatediff-cli-prompt-travel/data/video/dance33.mp4',
#    '/notebooks/storage/aj/animatediff-cli-prompt-travel/data/video/dance00001.mp4',
#    '/notebooks/storage/aj/animatediff-cli-prompt-travel/data/video/dance00002.mp4',
#    '/notebooks/storage/aj/animatediff-cli-prompt-travel/data/video/dance00003.mp4',
#     '/notebooks/storage/aj/animatediff-cli-prompt-travel/data/video/dance00004.mp4',
]
configs = [
#    '/notebooks/storage/aj/animatediff-cli-prompt-travel/config/fix/kobeni.json',
#    '/notebooks/storage/aj/animatediff-cli-prompt-travel/config/fix/real_base.json',
#    '/notebooks/storage/aj/animatediff-cli-prompt-travel/config/fix/just_anime.json',
#    '/notebooks/storage/aj/animatediff-cli-prompt-travel/config/fix/sukuna.json',
#    '/notebooks/storage/aj/animatediff-cli-prompt-travel/config/fix/todo.json',
#    '/notebooks/storage/aj/animatediff-cli-prompt-travel/config/fix/nanamin.json',
#    '/notebooks/storage/aj/animatediff-cli-prompt-travel/config/fix/jotaro.json',
    '/notebooks/storage/aj/animatediff-cli-prompt-travel/config/fix/real_base2.json',
]
#bg_config = '/notebooks/storage/aj/animatediff-cli-prompt-travel/config/fix/bg/BeachLCM.json'
bg_config = None

delete_if_exists = False
is_test = True
is_refine = False
############################################################
from animatediff.execute import execute
from pathlib import Path

repo_storage_dir_aj = Path('/storage/aj')
anime_diff_path = repo_storage_dir_aj / 'animatediff-cli-prompt-travel'

%cd "{Path(repo_storage_dir_aj, 'animatediff-cli-prompt-travel')}"

execute(videos=videos, configs=configs,urls=urls, delete_if_exists=delete_if_exists, is_test=is_test, is_refine=is_refine, bg_config=bg_config)

end_time = time.time()

execution_time = end_time - start_time
print(f"実行時間: {execution_time}秒")

/storage/aj/animatediff-cli-prompt-travel
[TikTok] Extracting URL: https://www.tiktok.com/@fukada0318/video/7290017445211819266
[TikTok] 7290017445211819266: Downloading video feed
[info] 7290017445211819266: Downloading 1 format(s): bytevc1_1080p_2508203-2
[download] ./data/video/dance00023.mp4 has already been downloaded
[download] 100% of    6.75MiB
impl
video1: ./data/video/dance00023.mp4
config already exists. skip create-config
Using generation config: stylize/dance00023/fg_00_dance00023/prompt.json
is_sdxl=False
is_v2=True
Using base model: runwayml/stable-diffusion-v1-5
Will save outputs to ./stylize/dance00023/fg_00_dance00023/2023-12-18_13-25-88


Preprocessing images (controlnet_openpose):   0%|          | 0/140 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(



Preprocessing images (controlnet_openpose): 100%|██████████| 140/140 [00:15<00:00,  9.23it/s]

Pipeline already loaded, skipping initialization
loading c='animatediff_controlnet' model


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:2025: UserWarning: for conv_in.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(f'for {key}: copying from a non-meta parameter in the checkpoint to a meta '

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:2025: UserWarning: for conv_in.bias: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(f'for {key}: copying from a non-meta parameter in the checkpoint to a meta '

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:2025: U

RuntimeError: "LayerNormKernelImpl" not implemented for 'Half'

In [2]:
from animatediff.front import launch

launch()

Preprocessing images (controlnet_openpose)  11% ━━━╺━━━━━━━━━━━━━━━━━━━━━━━━ 15/140  [ 0:00:34 < 0:04:24 , 0 it/s ]

14:21:26 INFO     Checking motion module...                                                         ]8;id=121625;file:///storage/aj/animatediff-cli-prompt-travel/src/animatediff/generate.py\generate.py]8;;\:]8;id=169454;file:///storage/aj/animatediff-cli-prompt-travel/src/animatediff/generate.py#612\612]8;;\

         INFO     Loading tokenizer...                                                              ]8;id=394701;file:///storage/aj/animatediff-cli-prompt-travel/src/animatediff/generate.py\generate.py]8;;\:]8;id=752427;file:///storage/aj/animatediff-cli-prompt-travel/src/animatediff/generate.py#636\636]8;;\

         INFO     Loading text encoder...                                                           ]8;id=429254;file:///storage/aj/animatediff-cli-prompt-travel/src/animatediff/generate.py\generate.py]8;;\:]8;id=54930;file:///storage/aj/animatediff-cli-prompt-travel/src/animatediff/generate.py#638\638]8;;\

14:21:34 INFO     Loading VAE...                                                                    ]8;id=54579;file:///storage/aj/animatediff-cli-prompt-travel/src/animatediff/generate.py\generate.py]8;;\:]8;id=82172;file:///storage/aj/animatediff-cli-prompt-travel/src/animatediff/generate.py#640\640]8;;\

...

create LoRA network from weights
skipped lora_te_text_model_encoder_layers_0_self_attn_k_proj (not found in modules_dim)
skipped lora_te_text_model_encoder_layers_0_self_attn_v_proj (not found in modules_dim)
skipped lora_te_text_model_encoder_layers_0_self_attn_q_proj (not found in modules_dim)
skipped lora_te_text_model_encoder_layers_0_self_attn_out_proj (not found in modules_dim)
skipped lora_te_text_model_encoder_layers_0_mlp_fc1 (not found in modules_dim)
skipped lora_te_text_model_encoder_layers_0_mlp_fc2 (not found in modules_dim)
skipped lora_te_text_model_encoder_layers_1_self_attn_k_proj (not found in modules_dim)
skipped lora_te_text_model_encoder_layers_1_self_attn_v_proj (not found in modules_dim)
skipped lora_te_text_model_encoder_layers_1_self_attn_q_proj (not found in modules_dim)
skipped lora_te_text_model_encoder_layers_1_self_attn_out_proj (not found in modules_dim)
skipped lora_te_text_model_encoder_layers_1_mlp_fc1 (not found in modules_dim)
skipped lora_te_text_m

14:24:33 INFO     No TI embeddings found                                                                  ]8;id=655712;file:///storage/aj/animatediff-cli-prompt-travel/src/animatediff/pipelines/ti.py\ti.py]8;;\:]8;id=232265;file:///storage/aj/animatediff-cli-prompt-travel/src/animatediff/pipelines/ti.py#104\104]8;;\

14:24:35 INFO     loading c='animatediff_controlnet' model                                          ]8;id=199815;file:///storage/aj/animatediff-cli-prompt-travel/src/animatediff/generate.py\generate.py]8;;\:]8;id=452037;file:///storage/aj/animatediff-cli-prompt-travel/src/animatediff/generate.py#783\783]8;;\

14:24:49 INFO     loading c='controlnet_openpose' model                                             ]8;id=124654;file:///storage/aj/animatediff-cli-prompt-travel/src/animatediff/generate.py\generate.py]8;;\:]8;id=127945;file:///storage/aj/animatediff-cli-prompt-travel/src/animatediff/generate.py#783\783]8;;\

KeyboardInterrupt: 

In [12]:
import gradio as gr
from animatediff.execute import execute
import sys
import io

# Define the function signature
def execute_wrapper(configs: list, urls: str, delete_if_exists: bool, is_test: bool, is_refine: bool) -> str:
    # Split the inputs by commas and strip whitespace
#    videos = [video.strip() for video in videos.split('\n') if video]
#    configs = [config.strip() for config in configs.split('\n') if config]
    config_paths = [os.path.join("/storage/aj/animatediff-cli-prompt-travel/config/fix", config + ".json") for config in configs]
    videos = []
    urls = [url.strip() for url in urls.split('\n') if url]
#    bg_config = bg_config.strip()
    bg_config = 'NA'
    # Redirect stdout to a string stream
    old_stdout = sys.stdout
    new_stdout = io.StringIO()
    sys.stdout = new_stdout
    
    try:
        # Execute the function with the inputs
        execute(videos=videos, configs=config_paths, urls=urls, delete_if_exists=delete_if_exists, is_test=is_test, is_refine=is_refine, bg_config=bg_config)
    finally:
        # Get the console log
        console_log = new_stdout.getvalue()
        
        # Restore stdout
        sys.stdout = old_stdout
    
    return console_log


# 指定したフォルダ以下のファイル一覧を取得
folder_path = "/storage/aj/animatediff-cli-prompt-travel/config/fix"
config_files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

# ファイル名から拡張子を取り除く
config_files_without_extension = [os.path.splitext(file)[0] for file in config_files]

# ファイル一覧をリストボックスの選択肢に変換
file_choices = [(file, file) for file in config_files_without_extension]


# Define the Gradio interface
iface = gr.Interface(
    execute_wrapper, 
    [
#        gr.Textbox(lines=3, placeholder="Enter video file paths, separated by commas", label="Videos"),
        gr.CheckboxGroup(file_choices, info="please select", label="Configs"),
#        gr.Textbox(lines=3, placeholder="Enter config file paths, separated by commas", label="Configs"),
        gr.Textbox(lines=3, placeholder="Enter URLs, separated by commas", label="URLs"),
#        gr.Textbox(lines=1, placeholder="Enter bg_config file path", label="BG Config"),
        gr.Checkbox(label="Delete if exists"),
        gr.Checkbox(label="Is test", value=True),
        gr.Checkbox(label="Is refine"),
    ],
    outputs="text"
)

# Launch the interface
iface.launch(share=True)

# 無限ループを追加してセルが終了しないようにする
while True:
    pass


  80% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━ 12/15  [ 0:18:39 < 0:00:11 , 0 it/s ]

In [ ]:
#!animatediff refine /storage/comfyUI/ComfyUI/output/2023-12-11/ADF/094059 -c /storage/aj/animatediff-cli-prompt-travel/stylize/jojo-dance00017/fg_00_jojo/prompt.json -W 768
!animatediff stylize composite /storage/aj/animatediff-cli-prompt-travel/stylize/real_base-dance00022 --simple_composite



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/usr/local/lib/python3.10/dist-packages/controlnet_aux/segment_anything/modeling/tiny_vit_sam.py:654: UserWarning: Overwriting tiny_vit_5m_224 in registry with controlnet_aux.segment_anything.modeling.tiny_vit_sam.tiny_vit_5m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  return register_model(fn_wrapper)
/usr/local/lib/python3.10/dist-packages/controlnet_aux/segment_anything/modeling/tiny_vit_sam.py:654: UserWarning: Overwriting tiny_vit_11m_224 in registry with controlnet_aux.segment_anything.modeling.tiny_vit_sam.tiny_vit_11m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  return register_model(fn_wrapper)
/usr/local/lib/python3.10/dist-packages/controlnet_aux/segment_anything/modeling/tiny_vit_sam.py:654: UserWarning: Overwriting tiny_vit_21m_224 in registry with controlnet_aux.segment_anything.modeling.tiny_vit_sam.tiny_vit_21m_224. This 

In [ ]:
#!cp -r /notebooks/storage/comfyUI/ComfyUI/output/2023-12-11/ADF/094059/* /notebooks/storage/aj/animatediff-cli-prompt-travel/stylize/jojo-dance00018/face_detailed


#!cd /notebooks/storage/aj/animatediff-cli-prompt-travel/stylize/jojo-dance00018/face_detailed && for file in _648359646536275_*; do newname=$(echo $file | sed 's/_648359646536275_//; s/_0*/_/'); mv "$file" "$newname"; done

#!cd /notebooks/storage/aj/animatediff-cli-prompt-travel/stylize/jojo-dance00018/face_detailed && for file in 000*\..png; do newname=$(echo $file | sed 's/_/./'); mv "$file" "$newname"; done

#!animatediff stylize composite /storage/aj/animatediff-cli-prompt-travel/stylize/jojo-dance00018 --simple_composite
!animatediff refine /storage/aj/animatediff-cli-prompt-travel/stylize/jojo-dance00020/fg_00_jojo/2023-12-11_16-58_00/00-2023-12-11_17-07 -W 768


/usr/local/lib/python3.10/dist-packages/controlnet_aux/segment_anything/modeling/tiny_vit_sam.py:654: UserWarning: Overwriting tiny_vit_5m_224 in registry with controlnet_aux.segment_anything.modeling.tiny_vit_sam.tiny_vit_5m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  return register_model(fn_wrapper)
/usr/local/lib/python3.10/dist-packages/controlnet_aux/segment_anything/modeling/tiny_vit_sam.py:654: UserWarning: Overwriting tiny_vit_11m_224 in registry with controlnet_aux.segment_anything.modeling.tiny_vit_sam.tiny_vit_11m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  return register_model(fn_wrapper)
/usr/local/lib/python3.10/dist-packages/controlnet_aux/segment_anything/modeling/tiny_vit_sam.py:654: UserWarning: Overwriting tiny_vit_21m_224 in registry with controlnet_aux.segment_anything.modeling.tiny_vit_sam.tiny_vit_21m_224. This 

In [5]:

from IPython.display import Video

# 透明な動画ファイルのパス
transparent_video_path = '/notebooks/storage/aj/animatediff-cli-prompt-travel/stylize/jjj-dance31/fg_00_jjj/2023-11-29_23-27_dance31.mp4_00/00_2023-11-29_23-30_.mp4'

# 透明な動画を再生
Video(transparent_video_path)

In [2]:
#!animatediff stylize create-config /storage/animatediff/animatediff-cli-prompt-travel/data/video/Download.mp4 --fps 25

# [3] generate mask
#!animatediff stylize create-mask /storage/animatediff/animatediff-cli-prompt-travel/stylize/2023-11-24T02-28-29-mumei-pyrite_v4

# The foreground is output to the following directory (FG_STYLYZE_DIR)
# STYLYZE_DIR/fg_00_timestamp_str
# The background is output to the following directory (BG_STYLYZE_DIR)
# STYLYZE_DIR/bg_timestamp_str

#!cp -r /notebooks/storage/aj/animatediff-cli-prompt-travel/stylize/Santa-dance00004/00_controlnet_image/controlnet_tile/* /notebooks/storage/aj/animatediff-cli-prompt-travel/stylize/Santa-dance00004/fg_00_Santa/00_controlnet_image/animatediff_controlnet
#!cp -r /notebooks/storage/animatediff/animatediff-cli-prompt-travel/stylize/2023-11-06T14-46-41-sample-majicmixrealistic_betterv2v25/00_controlnet_image/controlnet_tile/* /notebooks/storage/animatediff/animatediff-cli-prompt-travel/stylize/2023-11-06T14-46-41-sample-majicmixrealistic_betterv2v25/00_controlnet_image/controlnet_openpose
#!cp -r /notebooks/storage/animatediff/animatediff-cli-prompt-travel/stylize/2023-11-05T23-24-25-sample-majicmixrealistic_betterv2v25/00_controlnet_image/controlnet_tile/* /notebooks/storage/animatediff/animatediff-cli-prompt-travel/stylize/2023-11-05T23-24-25-sample-majicmixrealistic_betterv2v25/00_controlnet_image/controlnet_openpose

# [4] generate foreground

!animatediff stylize generate /storage/aj/animatediff-cli-prompt-travel/stylize/real_base2-dance00025/fg_00_real_base2 -L 16
#!rm -r /notebooks/storage/animatediff/animatediff-cli-prompt-travel/stylize/2023-11-09T14-08-07-e-majicmixrealistic_betterv2v25/2023*
#!animatediff stylize generate /notebooks/storage/animatediff/animatediff-cli-prompt-travel/stylize/2023-11-09T14-08-07-e-majicmixrealistic_betterv2v25 -L 16

#!animatediff refine /storage/animatediff/animatediff-cli-prompt-travel/stylize/2023-11-05T23-24-25-sample-majicmixrealistic_betterv2v25/2023-11-07T17-35-46_00/00-341774366206100 -W 768
#!animatediff refine /storage/animatediff/animatediff-cli-prompt-travel/stylize/2023-11-07T16-42-20-sample-majicmixrealistic_betterv2v25/2023-11-07T18-24-43_00/00-341774366206100 -W 768

#!animatediff stylize create-region /notebooks/storage/animatediff/animatediff-cli-prompt-travel/stylize/2023-11-07T16-42-20-sample-majicmixrealistic_betterv2v25

#!animatediff refine /storage/animatediff/animatediff-cli-prompt-travel/stylize/2023-11-09T06-42-55-dance22-majicmixrealistic_betterv2v25/2023-11-09T09-41-58_00/00-341774366206100 -W 768
#!animatediff refine /storage/animatediff/animatediff-cli-prompt-travel/stylize/2023-11-09T06-42-55-dance22-majicmixrealistic_betterv2v25/2023-11-09T09-41-58_01/00-341774366206100 -W 1024

#!animatediff refine /storage/aj/animatediff-cli-prompt-travel/stylize/jjj-dance33/fg_00_jjj/2023-11-29_01-33_00/00-2023-11-29_01-34 -W 512


#!rm -r /notebooks/storage/animatediff/animatediff-cli-prompt-travel/data/bg_work/*
#!for file in /notebooks/storage/animatediff/animatediff-cli-prompt-travel/stylize/2023-11-08T15-15-10-d10_msk-majicmixrealistic_betterv2v25/bg_2023-11-08T15-23-08/00_controlnet_image/controlnet_tile/*; do cp "/notebooks/storage/animatediff/animatediff-cli-prompt-travel/data/bg/90000001.png" "/notebooks/storage/animatediff/animatediff-cli-prompt-travel/data/bg_work/$(basename "$file")"; done
#!animatediff stylize composite /notebooks/storage/aj/animatediff-cli-prompt-travel/stylize/Santa-dance00004

/usr/local/lib/python3.10/dist-packages/controlnet_aux/segment_anything/modeling/tiny_vit_sam.py:654: UserWarning: Overwriting tiny_vit_5m_224 in registry with controlnet_aux.segment_anything.modeling.tiny_vit_sam.tiny_vit_5m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  return register_model(fn_wrapper)
/usr/local/lib/python3.10/dist-packages/controlnet_aux/segment_anything/modeling/tiny_vit_sam.py:654: UserWarning: Overwriting tiny_vit_11m_224 in registry with controlnet_aux.segment_anything.modeling.tiny_vit_sam.tiny_vit_11m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  return register_model(fn_wrapper)
/usr/local/lib/python3.10/dist-packages/controlnet_aux/segment_anything/modeling/tiny_vit_sam.py:654: UserWarning: Overwriting tiny_vit_21m_224 in registry with controlnet_aux.segment_anything.modeling.tiny_vit_sam.tiny_vit_21m_224. This 